In [1]:
import sys
sys.path.append("./models")
import random
from schnapsen.bots import RandBot
from schnapsen.bots import RdeepBot
from ML_bot import MLPlayingBot
from Deepbot import DeepLearningBot
from schnapsen.bots import BullyBot
import pathlib

from ML_bot import create_replay_memory_dataset

model_dir = "ML_models"
model_name = "simple_model"
rng = random.Random(42)
model_location = pathlib.Path(model_dir) / model_name

bot1 = RandBot(rng)
bot2 = RdeepBot(num_samples=20, depth=5, rand=rng)
bot3 = MLPlayingBot(model_location)
bot4 = BullyBot(rng)

create_replay_memory_dataset(bot1=bot1, bot2=bot3, num_of_games = 20000, parallel=True)


Game 1000 completed.Game 3500 completed.

Game 2000 completed.Game 4500 completed.

Game 500 completed.Game 3000 completed.

Game 1500 completed.
Game 4000 completed.
Game 5000 completed.
Game 2500 completed.
Game 6000 completed.
Game 8500 completed.
Game 7000 completed.
Game 9500 completed.
Game 5500 completed.
Game 8000 completed.
Game 6500 completed.
Game 9000 completed.
Game 7500 completed.
Game 10000 completed.
Game 11000 completed.
Game 13500 completed.
Game 12000 completed.
Game 10500 completed.Game 14500 completed.

Game 13000 completed.
Game 11500 completed.
Game 14000 completed.
Game 12500 completed.
Game 16000 completed.
Game 15000 completed.
Game 18500 completed.
Game 17000 completed.
Game 19500 completed.
Game 15500 completed.
Game 18000 completed.
Game 16500 completed.
Game 19000 completed.
Game 17500 completed.
Game 20000 completed.


In [ ]:
import sys
sys.path.append("./models")

from ML_bot import train_model

train_model("NN")

In [ ]:
import pathlib
import random

from schnapsen.game import SchnapsenGamePlayEngine
from schnapsen.bots import RandBot

from ML_bot import MLPlayingBot

engine = SchnapsenGamePlayEngine()
model_dir = "ML_models"
model_name = "simple_model"
model_location = pathlib.Path(model_dir) / model_name

bot1 = MLPlayingBot(model_location, name="MLBot")
bot2 = RandBot(rng, "RandBot")

winner_id, game_points, score = engine.play_game(bot1, bot2, random.Random(0))
print(f"Game ended. Winner is {winner_id._Bot__name} with {game_points} points and {score}")

In [3]:
import sys
sys.path.append("./models")
from ML_bot import MLPlayingBot
from schnapsen.bots import RdeepBot
from schnapsen.bots import RandBot
from Deepbot import DeepLearningBot
from schnapsen.game import SchnapsenGamePlayEngine
import pathlib
import random


engine = SchnapsenGamePlayEngine()
model_dir = "ML_models"
model_name = "simple_model"
model_location = pathlib.Path(model_dir) / model_name

model_path = "./models/model_20250112_010939_epochs50_batch32_lr0.001.pt"

rng = random.Random(42)

bot1 = MLPlayingBot(model_location, name="MLBot")
bot2 = RandBot(rng, "RandBot")
bot3 = RdeepBot(num_samples=5, depth=2, rand=rng, name="RdeepBot")
bot4 = DeepLearningBot(model_path=model_path, input_size=173, hidden_size=64, name="DLbot")

def round_robin_tournament():
    wins_MLBot = 0
    wins_RDeep = 0
    wins_Randy = 0
    for i in range (500):

        if i % 500 == 0:
            print(f"Current progress: game {i}")

        winner_id, game_points, score = engine.play_game(bot1, bot4, random.Random(i))

        if winner_id._Bot__name == "DLbot":
            wins_MLBot +=1
        elif winner_id._Bot__name == "RdeepBot":
            wins_RDeep +=1
        else:
            wins_Randy +=1

        winner_id2, game_points2, score2 = engine.play_game(bot1, bot3, random.Random(i))

        if winner_id2._Bot__name == "DLbot":
            wins_MLBot +=1
        elif winner_id2._Bot__name == "RdeepBot":
            wins_RDeep +=1
        else:
            wins_Randy +=1

        winner_id3, game_points3, score3 = engine.play_game(bot4, bot3, random.Random(i))

        if winner_id3._Bot__name == "DLbot":
            wins_MLBot +=1
        elif winner_id3._Bot__name == "RdeepBot":
            wins_RDeep +=1
        else:
            wins_Randy +=1



    print (f"DLBot: {wins_MLBot}")
    print (f"Randy: {wins_Randy}")
    print (f"Rdeep: {wins_RDeep}")

round_robin_tournament()

Current progress: game 0
DLBot: 311
Randy: 548
Rdeep: 641


In [1]:
import sys
sys.path.append("./models")
from Deepbot import train_DL_model, gpu_check

# gpu_check()

data_file = "./ML_replay_memories/replay_memory.txt"  # Replace with your data file path
output_model_path = "./models"
input_dim = 173
hidden_dim = 64
train_DL_model(data_file, output_model_path, input_dim, hidden_dim)

Training on: cuda
Starting training...
Epoch 1/50, Loss: 0.6035


KeyboardInterrupt: 